# Import Required Packages

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#!pip install torchvision
#!pip install opencv-python

# Download Files

In [3]:
#training controls
batch_size = 64
epochs = 10
training_size = 0.8
learning_rate = 0.0001

# input image dimensions
img_rows, img_cols = 268, 182

In [4]:
# data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [5]:
# opening the dataset
dataset = csv.reader(open("MovieGenre417.csv",encoding="utf8",errors='replace'), delimiter=",")

# skipping the header line
next(dataset)

['', 'index', 'imdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [6]:
# extract images from zip folder

import zipfile as zf

files = zf.ZipFile("FinalProjectUp.zip", 'r')
files.extractall()
files.close()

In [7]:
# list of image files in SampleMoviePosters folder
flist=glob.glob('FinalProject/*.jpg')  

In [8]:
len(flist)

5384

In [9]:
image_ids = []

for path in flist:
    start = path.rfind("/")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [10]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre417.csv")
dataset2

,Unnamed: 0,index,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,0,0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,2,2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
2,11,11,112896,http://www.imdb.com/title/tt112896,Dracula: Dead and Loving It (1995),5.8,Comedy|Fantasy|Horror,https://images-na.ssl-images-amazon.com/images...
3,23,23,114168,http://www.imdb.com/title/tt114168,Powder (1995),6.5,Drama|Fantasy|Mystery,https://images-na.ssl-images-amazon.com/images...
4,26,26,114011,http://www.imdb.com/title/tt114011,Now and Then (1995),6.8,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...,...,...
5379,39322,40060,4508542,http://www.imdb.com/title/tt4508542,Nacida para ganar (2016),4.8,Comedy,https://images-na.ssl-images-amazon.com/images...
5380,39333,40071,1018706,http://www.imdb.com/title/tt1018706,CÌ£o Sem Dono (2007),6.9,Drama,https://images-na.ssl-images-amazon.com/images...
5381,39347,40085,4189294,http://www.imdb.com/title/tt4189294,Lego DC Comics: Batman Be-Leaguered (2014),6.8,Animation|Short|Action,https://images-na.ssl-images-amazon.com/images...
5382,39362,40100,98216,http://www.imdb.com/title/tt98216,Roller Blade Warriors: Taken by Force (1989),3.9,Action|Adventure|Sci-Fi,https://images-na.ssl-images-amazon.com/images...


# Data Preprocessing

In [11]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    #print(dataset2["imdbId"])
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [12]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df = y_df[['Drama', 'Comedy', 'Romance', 'Action', 'Crime']]
classes = y_df.columns.tolist()
y_df

,Drama,Comedy,Romance,Action,Crime
2371315,1,0,0,0,1
2504404,1,0,0,0,1
363473,1,0,0,0,0
389778,1,0,0,0,0
42693,0,1,0,1,0
...,...,...,...,...,...
409904,0,0,0,1,0
106685,1,1,0,0,0
3375370,0,1,0,0,0
3331846,1,1,0,0,0


In [13]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,Unnamed: 0,index,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,0,0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
1,2,2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 1, 0, 0]"
2,11,11,112896,http://www.imdb.com/title/tt112896,Dracula: Dead and Loving It (1995),5.8,Comedy|Fantasy|Horror,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
3,23,23,114168,http://www.imdb.com/title/tt114168,Powder (1995),6.5,Drama|Fantasy|Mystery,https://images-na.ssl-images-amazon.com/images...,"[1, 0, 0, 0, 0]"
4,26,26,114011,http://www.imdb.com/title/tt114011,Now and Then (1995),6.8,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[1, 1, 1, 0, 0]"
...,...,...,...,...,...,...,...,...,...
5379,39322,40060,4508542,http://www.imdb.com/title/tt4508542,Nacida para ganar (2016),4.8,Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 1, 0, 0, 0]"
5380,39333,40071,1018706,http://www.imdb.com/title/tt1018706,CÌ£o Sem Dono (2007),6.9,Drama,https://images-na.ssl-images-amazon.com/images...,"[1, 0, 0, 0, 0]"
5381,39347,40085,4189294,http://www.imdb.com/title/tt4189294,Lego DC Comics: Batman Be-Leaguered (2014),6.8,Animation|Short|Action,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 1, 0]"
5382,39362,40100,98216,http://www.imdb.com/title/tt98216,Roller Blade Warriors: Taken by Force (1989),3.9,Action|Adventure|Sci-Fi,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 1, 0]"


In [14]:
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

#tempY

# Train/Test Split

In [15]:
#setting the length of training data
length=int(len(flist)*training_size)
length

4307

In [16]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('/')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [17]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [18]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [19]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (4307, 3, 268, 182)
4307 train samples
1077 test samples


In [20]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [21]:
# Metric calculation

def metric(scores, targets):
    """
    :param scores: the output the model predict
    :param targets: the gt label
    :return: OP, OR, OF1, CP, CR, CF1
    calculate the Precision of every class by: TP/TP+FP i.e. TP/total predict
    calculate the Recall by: TP/total GT
    """
    num, num_class = scores.shape
    gt_num = np.zeros(num_class)
    tp_num = np.zeros(num_class)
    predict_num = np.zeros(num_class)


    for index in range(num_class):
        score = scores[:, index]
        target = targets[:, index]

        gt_num[index] = np.sum(target == 1)
        predict_num[index] = np.sum(score >= 0.5)
        tp_num[index] = np.sum(target * (score >= 0.5))

    predict_num[predict_num == 0] = 1  # avoid dividing 0
    OP = np.sum(tp_num) / np.sum(predict_num) #OP (Overall Precision) is the ratio of the number of correctly predicted positive samples to the total number of positive predictions made by the model
    OR = np.sum(tp_num) / np.sum(gt_num) #OR (Overall Recall) is the ratio of the number of correctly predicted positive samples to the total number of positive samples in the ground truth.
    OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.

    return OP, OR, OF1

# Model 1: Resnet50

In [22]:
# Resnet50 model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.resnet(x)
        x = self.dropout(x)
        
        return x

In [23]:
model = ResNet()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
from statistics import mean
import numpy as np

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)

        preds = torch.round(output)
            
        #acc_list = []
        #preds = torch.round(output)
        #for i in range(len(preds)):
        #    result = 0
        #    denom = 0
        #    for j in range(len(classes)):
        #        if target[i][j] == 1 or preds[i][j] == 1:
        #            denom += 1
        #            if preds[i][j] == target[i][j]:
        #                result+=1
        #    acc_list.append(result/denom)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        loss_lst_train.append(loss.data.item())
        OF1_lst_train.append(OF1)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model.eval()
    test_loss = 0
    i = 0
    
    OP_final = 0
    OR_final = 0
    OF1_final = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        OP_final += OP
        OR_final += OR
        OF1_final += OF1
        
    loss_lst_test.append(test_loss.data.item()/i)
    OF1_lst_test.append(OF1_final/i)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP_final/i, OR_final/i, OF1_final/i))

loss_lst_train = []
OF1_lst_train = []

loss_lst_test = []
OF1_lst_test = []

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/4307 (0%)]	Loss: 0.663779 	OP: 0.464286	OR: 0.128713	OF1: 0.201550
Train Epoch: 0 [64/4307 (1%)]	Loss: 0.611307 	OP: 0.727273	OR: 0.155340	OF1: 0.256000
Train Epoch: 0 [128/4307 (3%)]	Loss: 0.629323 	OP: 0.521739	OR: 0.123711	OF1: 0.200000
Train Epoch: 0 [192/4307 (4%)]	Loss: 0.604056 	OP: 0.388889	OR: 0.081395	OF1: 0.134615
Train Epoch: 0 [256/4307 (6%)]	Loss: 0.611843 	OP: 0.760000	OR: 0.193878	OF1: 0.308943
Train Epoch: 0 [320/4307 (7%)]	Loss: 0.574073 	OP: 0.684211	OR: 0.142857	OF1: 0.236364
Train Epoch: 0 [384/4307 (9%)]	Loss: 0.590977 	OP: 0.680000	OR: 0.161905	OF1: 0.261538
Train Epoch: 0 [448/4307 (10%)]	Loss: 0.634496 	OP: 0.666667	OR: 0.117647	OF1: 0.200000
Train Epoch: 0 [512/4307 (12%)]	Loss: 0.625372 	OP: 0.571429	OR: 0.116505	OF1: 0.193548
Train Epoch: 0 [576/4307 (13%)]	Loss: 0.645597 	OP: 0.700000	OR: 0.126126	OF1: 0.213740
Train Epoch: 0 [640/4307 (15%)]	Loss: 0.578532 	OP: 0.772727	OR: 0.170000	OF1: 0.278689
Train Epoch: 0 [704/4307 (16%)]	Loss: 0.58

<ipython-input-25-9fe02c90224d>:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.5336 
OP: 0.689040
OR: 0.196484
OF1: 0.304910

Train Epoch: 1 [0/4307 (0%)]	Loss: 0.523886 	OP: 0.833333	OR: 0.309278	OF1: 0.451128
Train Epoch: 1 [64/4307 (1%)]	Loss: 0.558500 	OP: 0.703704	OR: 0.188119	OF1: 0.296875
Train Epoch: 1 [128/4307 (3%)]	Loss: 0.553790 	OP: 0.720000	OR: 0.178218	OF1: 0.285714
Train Epoch: 1 [192/4307 (4%)]	Loss: 0.528563 	OP: 0.928571	OR: 0.254902	OF1: 0.400000
Train Epoch: 1 [256/4307 (6%)]	Loss: 0.545514 	OP: 0.705882	OR: 0.255319	OF1: 0.375000
Train Epoch: 1 [320/4307 (7%)]	Loss: 0.516800 	OP: 0.862069	OR: 0.240385	OF1: 0.375940
Train Epoch: 1 [384/4307 (9%)]	Loss: 0.502045 	OP: 0.846154	OR: 0.358696	OF1: 0.503817
Train Epoch: 1 [448/4307 (10%)]	Loss: 0.546841 	OP: 0.758621	OR: 0.229167	OF1: 0.352000
Train Epoch: 1 [512/4307 (12%)]	Loss: 0.523004 	OP: 0.777778	OR: 0.223404	OF1: 0.347107
Train Epoch: 1 [576/4307 (13%)]	Loss: 0.540657 	OP: 0.814815	OR: 0.213592	OF1: 0.338462
Train Epoch: 1 [640/4307 (15%)]	Loss: 0.571775 	OP: 

# Model 2: DenseNet201

In [22]:
import torchvision
import torch.nn as nn

model2 = torchvision.models.densenet201(pretrained=True)
num_ftrs = model2.classifier.in_features
model2.classifier = nn.Linear(num_ftrs, len(classes))

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
from statistics import mean
import numpy as np

def train(epoch):
    model2.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model2(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        loss_lst_train2.append(loss.data.item())
        OF1_lst_train2.append(OF1)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model2.eval()
    test_loss = 0
    i = 0
    
    OP_final = 0
    OR_final = 0
    OF1_final = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model2(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        OP_final += OP
        OR_final += OR
        OF1_final += OF1
        
    loss_lst_test2.append(test_loss.data.item()/i)
    OF1_lst_test2.append(OF1_final/i)
        
    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP, OR, OF1))

loss_lst_train2 = []
OF1_lst_train2 = []

loss_lst_test2 = []
OF1_lst_test2 = []    
    
for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/4307 (0%)]	Loss: 0.685201 	OP: 0.333333	OR: 0.028846	OF1: 0.053097
Train Epoch: 0 [64/4307 (1%)]	Loss: 0.666958 	OP: 0.416667	OR: 0.050000	OF1: 0.089286
Train Epoch: 0 [128/4307 (3%)]	Loss: 0.643762 	OP: 0.444444	OR: 0.038095	OF1: 0.070175
Train Epoch: 0 [192/4307 (4%)]	Loss: 0.613103 	OP: 0.666667	OR: 0.081633	OF1: 0.145455
Train Epoch: 0 [256/4307 (6%)]	Loss: 0.645037 	OP: 0.500000	OR: 0.086538	OF1: 0.147541
Train Epoch: 0 [320/4307 (7%)]	Loss: 0.617840 	OP: 0.500000	OR: 0.095238	OF1: 0.160000
Train Epoch: 0 [384/4307 (9%)]	Loss: 0.604458 	OP: 0.571429	OR: 0.121212	OF1: 0.200000
Train Epoch: 0 [448/4307 (10%)]	Loss: 0.622469 	OP: 0.555556	OR: 0.142857	OF1: 0.227273
Train Epoch: 0 [512/4307 (12%)]	Loss: 0.571185 	OP: 0.666667	OR: 0.156863	OF1: 0.253968
Train Epoch: 0 [576/4307 (13%)]	Loss: 0.586665 	OP: 0.700000	OR: 0.205882	OF1: 0.318182
Train Epoch: 0 [640/4307 (15%)]	Loss: 0.580106 	OP: 0.656250	OR: 0.203883	OF1: 0.311111
Train Epoch: 0 [704/4307 (16%)]	Loss: 0.56

<ipython-input-23-eea8f805c656>:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.4667 
OP: 0.688889
OR: 0.407895
OF1: 0.512397

Train Epoch: 1 [0/4307 (0%)]	Loss: 0.357526 	OP: 0.836364	OR: 0.450980	OF1: 0.585987
Train Epoch: 1 [64/4307 (1%)]	Loss: 0.382521 	OP: 0.862745	OR: 0.431373	OF1: 0.575163
Train Epoch: 1 [128/4307 (3%)]	Loss: 0.417747 	OP: 0.888889	OR: 0.444444	OF1: 0.592593
Train Epoch: 1 [192/4307 (4%)]	Loss: 0.381194 	OP: 0.909091	OR: 0.485437	OF1: 0.632911
Train Epoch: 1 [256/4307 (6%)]	Loss: 0.393850 	OP: 0.903846	OR: 0.456311	OF1: 0.606452
Train Epoch: 1 [320/4307 (7%)]	Loss: 0.328996 	OP: 0.950000	OR: 0.600000	OF1: 0.735484
Train Epoch: 1 [384/4307 (9%)]	Loss: 0.333777 	OP: 0.962963	OR: 0.509804	OF1: 0.666667
Train Epoch: 1 [448/4307 (10%)]	Loss: 0.373049 	OP: 0.854839	OR: 0.535354	OF1: 0.658385
Train Epoch: 1 [512/4307 (12%)]	Loss: 0.350237 	OP: 0.884615	OR: 0.489362	OF1: 0.630137
Train Epoch: 1 [576/4307 (13%)]	Loss: 0.351693 	OP: 0.909091	OR: 0.520833	OF1: 0.662252
Train Epoch: 1 [640/4307 (15%)]	Loss: 0.381123 	OP: 

# Model 3: ResNet18

In [24]:
from torchvision import models

class ResNet152(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet152, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.resnet(x)
        x = self.dropout(x)
        return x

In [25]:
model3 = ResNet152()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

/home/rjw8ng/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [26]:
from statistics import mean
import numpy as np

def train(epoch):
    model3.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model3(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        loss_lst_train3.append(loss.data.item())
        OF1_lst_train3.append(OF1)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model3.eval()
    test_loss = 0
    i = 0
    
    OP_final = 0
    OR_final = 0
    OF1_final = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model3(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        OP_final += OP
        OR_final += OR
        OF1_final += OF1
        
    loss_lst_test3.append(test_loss.data.item()/i)
    OF1_lst_test3.append(OF1_final/i)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP_final/i, OR_final/i, OF1_final/i))

loss_lst_train3 = []
OF1_lst_train3 = []

loss_lst_test3 = []
OF1_lst_test3 = []    

for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/4307 (0%)]	Loss: 0.750986 	OP: 0.402062	OR: 0.378641	OF1: 0.390000
Train Epoch: 0 [64/4307 (1%)]	Loss: 0.769544 	OP: 0.301205	OR: 0.260417	OF1: 0.279330
Train Epoch: 0 [128/4307 (3%)]	Loss: 0.712539 	OP: 0.426230	OR: 0.247619	OF1: 0.313253
Train Epoch: 0 [192/4307 (4%)]	Loss: 0.684681 	OP: 0.488372	OR: 0.198113	OF1: 0.281879
Train Epoch: 0 [256/4307 (6%)]	Loss: 0.636422 	OP: 0.730769	OR: 0.204301	OF1: 0.319328
Train Epoch: 0 [320/4307 (7%)]	Loss: 0.648396 	OP: 0.586207	OR: 0.175258	OF1: 0.269841
Train Epoch: 0 [384/4307 (9%)]	Loss: 0.629176 	OP: 0.666667	OR: 0.142857	OF1: 0.235294
Train Epoch: 0 [448/4307 (10%)]	Loss: 0.631451 	OP: 0.357143	OR: 0.049020	OF1: 0.086207
Train Epoch: 0 [512/4307 (12%)]	Loss: 0.621434 	OP: 0.727273	OR: 0.155340	OF1: 0.256000
Train Epoch: 0 [576/4307 (13%)]	Loss: 0.625002 	OP: 0.550000	OR: 0.110000	OF1: 0.183333
Train Epoch: 0 [640/4307 (15%)]	Loss: 0.638334 	OP: 0.733333	OR: 0.108911	OF1: 0.189655
Train Epoch: 0 [704/4307 (16%)]	Loss: 0.59

<ipython-input-26-22a4990e0b87>:53: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.5218 
OP: 0.750149
OR: 0.266637
OF1: 0.392530

Train Epoch: 1 [0/4307 (0%)]	Loss: 0.564273 	OP: 0.760000	OR: 0.182692	OF1: 0.294574
Train Epoch: 1 [64/4307 (1%)]	Loss: 0.556396 	OP: 0.736842	OR: 0.132075	OF1: 0.224000
Train Epoch: 1 [128/4307 (3%)]	Loss: 0.575893 	OP: 0.700000	OR: 0.150538	OF1: 0.247788
Train Epoch: 1 [192/4307 (4%)]	Loss: 0.553828 	OP: 0.777778	OR: 0.203883	OF1: 0.323077
Train Epoch: 1 [256/4307 (6%)]	Loss: 0.554175 	OP: 0.807692	OR: 0.205882	OF1: 0.328125
Train Epoch: 1 [320/4307 (7%)]	Loss: 0.540455 	OP: 0.733333	OR: 0.229167	OF1: 0.349206
Train Epoch: 1 [384/4307 (9%)]	Loss: 0.552138 	OP: 0.794118	OR: 0.272727	OF1: 0.406015
Train Epoch: 1 [448/4307 (10%)]	Loss: 0.566291 	OP: 0.709677	OR: 0.217822	OF1: 0.333333
Train Epoch: 1 [512/4307 (12%)]	Loss: 0.547407 	OP: 0.916667	OR: 0.222222	OF1: 0.357724
Train Epoch: 1 [576/4307 (13%)]	Loss: 0.554361 	OP: 0.846154	OR: 0.220000	OF1: 0.349206
Train Epoch: 1 [640/4307 (15%)]	Loss: 0.548095 	OP: 

# Custom CNN

In [24]:
import torch.nn as nn

class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(True)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7,7))
        self.classifier = nn.Sequential(
            nn.Linear(256*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, len(classes)),
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [25]:
model4 = CustomCNN()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model4.parameters(), lr=learning_rate)

In [26]:
from statistics import mean
import numpy as np

def train(epoch):
    model4.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model4(data)

        preds = torch.round(output)
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        loss_lst_train4.append(loss.data.item())
        OF1_lst_train4.append(OF1)
        
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} \tOP: {:.6f}\tOR: {:.6f}\tOF1: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), OP, OR, OF1))

def test():
    print('test')
    model4.eval()
    test_loss = 0
    i = 0
    
    OP_final = 0
    OR_final = 0
    OF1_final = 0
    
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        with torch.no_grad():
            data, target = Variable(data, volatile=True).float(), Variable(target).float()
            output = model4(data)
        
        preds = torch.round(output)
            
        loss = criterion(output, target)
        test_loss += loss
        
        target = target.detach().numpy()
        preds = preds.detach().numpy()
        OP, OR, OF1 = metric(preds, target)
        
        OP_final += OP
        OR_final += OR
        OF1_final += OF1
        
    loss_lst_test4.append(test_loss.data.item()/i)
    OF1_lst_test4.append(OF1_final/i)

    print('\nTest set: \nAverage sq_loss: {:.4f} \nOP: {:.6f}\nOR: {:.6f}\nOF1: {:.6f}\n'.format(test_loss.data.item()/i, OP_final/i, OR_final/i, OF1_final/i))

loss_lst_train4 = []
OF1_lst_train4 = []

loss_lst_test4 = []
OF1_lst_test4 = []    
    
for epoch in range(0, epochs):
    train(epoch)
    test()

<ipython-input-21-447663f93c68>:28: RuntimeWarning: invalid value encountered in double_scalars
  OF1 = (2 * OP * OR) / (OP + OR) #OF1 (Overall F1 Score) is the harmonic mean of precision and recall.


Train Epoch: 0 [0/4307 (0%)]	Loss: 0.693548 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [64/4307 (1%)]	Loss: 0.673958 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [128/4307 (3%)]	Loss: 0.651217 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [192/4307 (4%)]	Loss: 0.614288 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [256/4307 (6%)]	Loss: 0.575148 	OP: 0.000000	OR: 0.000000	OF1: nan
Train Epoch: 0 [320/4307 (7%)]	Loss: 0.549834 	OP: 0.606061	OR: 0.198020	OF1: 0.298507
Train Epoch: 0 [384/4307 (9%)]	Loss: 0.547184 	OP: 0.606061	OR: 0.396040	OF1: 0.479042
Train Epoch: 0 [448/4307 (10%)]	Loss: 0.515406 	OP: 0.632353	OR: 0.462366	OF1: 0.534161
Train Epoch: 0 [512/4307 (12%)]	Loss: 0.577373 	OP: 0.544118	OR: 0.389474	OF1: 0.453988
Train Epoch: 0 [576/4307 (13%)]	Loss: 0.504035 	OP: 0.655738	OR: 0.430108	OF1: 0.519481
Train Epoch: 0 [640/4307 (15%)]	Loss: 0.575263 	OP: 0.517857	OR: 0.284314	OF1: 0.367089
Train Epoch: 0 [704/4307 (16%)]	Loss: 0.562592 	OP: 0.357143	OR: 0.

<ipython-input-26-33e25302e374>:41: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.5366 
OP: 0.481925
OR: 0.062725
OF1: 0.110734

Train Epoch: 1 [0/4307 (0%)]	Loss: 0.519318 	OP: 0.600000	OR: 0.087379	OF1: 0.152542
Train Epoch: 1 [64/4307 (1%)]	Loss: 0.549761 	OP: 0.470588	OR: 0.079208	OF1: 0.135593
Train Epoch: 1 [128/4307 (3%)]	Loss: 0.570125 	OP: 0.368421	OR: 0.068627	OF1: 0.115702
Train Epoch: 1 [192/4307 (4%)]	Loss: 0.552091 	OP: 0.363636	OR: 0.080000	OF1: 0.131148
Train Epoch: 1 [256/4307 (6%)]	Loss: 0.536003 	OP: 0.363636	OR: 0.086022	OF1: 0.139130
Train Epoch: 1 [320/4307 (7%)]	Loss: 0.518619 	OP: 0.733333	OR: 0.106796	OF1: 0.186441
Train Epoch: 1 [384/4307 (9%)]	Loss: 0.554349 	OP: 0.461538	OR: 0.058252	OF1: 0.103448
Train Epoch: 1 [448/4307 (10%)]	Loss: 0.514676 	OP: 0.562500	OR: 0.092784	OF1: 0.159292
Train Epoch: 1 [512/4307 (12%)]	Loss: 0.556097 	OP: 0.733333	OR: 0.213592	OF1: 0.330827
Train Epoch: 1 [576/4307 (13%)]	Loss: 0.558576 	OP: 0.526316	OR: 0.096154	OF1: 0.162602
Train Epoch: 1 [640/4307 (15%)]	Loss: 0.542550 	OP: 